In [33]:
# Continue with regular imports
import matplotlib.pyplot as plt
import torch
import torchvision
from torch import nn
from torchvision import transforms
from torchinfo import summary
import os
import zipfile

from pathlib import Path

import requests

from GoingModular import data_setup, engine

device = "cuda" if torch.cuda.is_available() else "cpu"


In [2]:
# EfficientNet_B0_Weights 

weights = torchvision.models.EfficientNet_B0_Weights.DEFAULT 
# weights
auto_transforms=weights.transforms()
train_dataloader,test_dataloader,class_names=data_setup.create_dataloaders(train_dir=r'../data/pizza_steak_sushi/test',
                                                                           test_dir= r'../data/pizza_steak_sushi/train',
                                                                           transform=auto_transforms,
                                                                           batch_size=32)
train_dataloader,test_dataloader,class_names

model=torchvision.models.efficientnet_b0(weights=weights).to(device)

In [3]:
model.state_dict
summary(model=model,
        input_size=(32,3,224,224),
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"])

Layer (type (var_name))                                      Input Shape          Output Shape         Param #              Trainable
EfficientNet (EfficientNet)                                  [32, 3, 224, 224]    [32, 1000]           --                   True
├─Sequential (features)                                      [32, 3, 224, 224]    [32, 1280, 7, 7]     --                   True
│    └─Conv2dNormActivation (0)                              [32, 3, 224, 224]    [32, 32, 112, 112]   --                   True
│    │    └─Conv2d (0)                                       [32, 3, 224, 224]    [32, 32, 112, 112]   864                  True
│    │    └─BatchNorm2d (1)                                  [32, 32, 112, 112]   [32, 32, 112, 112]   64                   True
│    │    └─SiLU (2)                                         [32, 32, 112, 112]   [32, 32, 112, 112]   --                   --
│    └─Sequential (1)                                        [32, 32, 112, 112]   [32, 16, 112

We can freeze all of the layers/parameters in the features section by setting the attribute requires_grad=False
For parameters with requires_grad=False, PyTorch doesn't track gradient updates and in turn, these parameters won't be changed by our optimizer during training.

In essence, a parameter with requires_grad=False is "untrainable" or "frozen" in place.

In [4]:
for param in model.features.parameters():
    param.requires_grad = False

output_shape=len(class_names)

model.classifier=torch.nn.Sequential(
    torch.nn.Dropout(p=0.2,inplace=True),
    torch.nn.Linear(in_features=1280,
                    out_features=output_shape,
                    bias=True
                    )
)
# # Do a summary *after* freezing the features and changing the output classifier layer (uncomment for actual output)
summary(model, 
        input_size=(32, 3, 224, 224), # make sure this is "input_size", not "input_shape" (batch_size, color_channels, height, width)
        verbose=0,
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"])


Layer (type (var_name))                                      Input Shape          Output Shape         Param #              Trainable
EfficientNet (EfficientNet)                                  [32, 3, 224, 224]    [32, 3]              --                   Partial
├─Sequential (features)                                      [32, 3, 224, 224]    [32, 1280, 7, 7]     --                   False
│    └─Conv2dNormActivation (0)                              [32, 3, 224, 224]    [32, 32, 112, 112]   --                   False
│    │    └─Conv2d (0)                                       [32, 3, 224, 224]    [32, 32, 112, 112]   (864)                False
│    │    └─BatchNorm2d (1)                                  [32, 32, 112, 112]   [32, 32, 112, 112]   (64)                 False
│    │    └─SiLU (2)                                         [32, 32, 112, 112]   [32, 32, 112, 112]   --                   --
│    └─Sequential (1)                                        [32, 32, 112, 112]   [32, 

In [5]:
loss_fn=nn.CrossEntropyLoss()
optim=torch.optim.Adam(model.parameters(),lr=0.01)


from timeit import default_timer as Timer

star_time=Timer()

results=engine.train(model=model,
                     train_dataloader=train_dataloader,
                     test_dataloader=test_dataloader,
                     optimizer=optim,
                     loss_fn=loss_fn,
                     epochs=2,
                     device=device)
end_time=Timer()

print(f"[INFO] Total training time: {end_time-star_time:.3f} seconds")


  0%|          | 0/2 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 1.0751 | train_acc: 0.3911 | test_loss: 0.6065 | test_acc: 0.7344
Epoch: 2 | train_loss: 0.4276 | train_acc: 0.9186 | test_loss: 0.5571 | test_acc: 0.7812
[INFO] Total training time: 12.065 seconds


In [6]:
from torch.utils.tensorboard import SummaryWriter

writer=SummaryWriter()


modify standart train function used before to use SummaryWtiretr

In [23]:
def create_writer(experiment_name:str, model_name:str,extra:str=None):
    """Creates a torch.utils.tensorboard.writer.SummaryWriter() instance saving to a specific log_dir.

    log_dir is a combination of runs/timestamp/experiment_name/model_name/extra.

    Where timestamp is the current date in YYYY-MM-DD format.

    Args:
        experiment_name (str): Name of experiment.
        model_name (str): Name of model.
        extra (str, optional): Anything extra to add to the directory. Defaults to None.

    Returns:
        torch.utils.tensorboard.writer.SummaryWriter(): Instance of a writer saving to log_dir.

    Example usage:
        # Create a writer saving to "runs/2022-06-04/data_10_percent/effnetb2/5_epochs/"
        writer = create_writer(experiment_name="data_10_percent",
                               model_name="effnetb2",
                               extra="5_epochs")
        # The above is the same as:
        writer = SummaryWriter(log_dir="runs/2022-06-04/data_10_percent/effnetb2/5_epochs/")
    """
    from datetime import datetime
    import os

    # Get timestamp of current date (all experiments on certain day live in same folder)
    timestamp = datetime.now().strftime("%Y-%m-%d") # returns current date in YYYY-MM-DD format

    if extra:
        # Create log directory path
        log_dir = os.path.join("runs", timestamp, experiment_name, model_name, extra)
    else:
        log_dir = os.path.join("runs", timestamp, experiment_name, model_name)
        
    print(f"[INFO] Created SummaryWriter, saving to: {log_dir}...")
    return SummaryWriter(log_dir=log_dir)

In [24]:
from typing import Dict, List
from tqdm.auto import tqdm

from GoingModular.engine import train_step, test_step

def train(model: torch.nn.Module, 
          train_dataloader: torch.utils.data.DataLoader, 
          test_dataloader: torch.utils.data.DataLoader, 
          optimizer: torch.optim.Optimizer,
          loss_fn: torch.nn.Module,
          epochs: int,
          device: torch.device,
          writer: torch.utils.tensorboard.writer.SummaryWriter
          ) -> Dict[str, List]:

    results = {"train_loss": [],
               "train_acc": [],
               "test_loss": [],
               "test_acc": []
    }

    for epoch in tqdm(range(epochs)):
        train_loss, train_acc = train_step(model=model,
                                           dataloader=train_dataloader,
                                           loss_fn=loss_fn,
                                           optimizer=optimizer,
                                           device=device)
        test_loss, test_acc = test_step(model=model,
                                        dataloader=test_dataloader,
                                        loss_fn=loss_fn,
                                        device=device)

        # Print out what's happening
        print(
          f"Epoch: {epoch+1} | "
          f"train_loss: {train_loss:.4f} | "
          f"train_acc: {train_acc:.4f} | "
          f"test_loss: {test_loss:.4f} | "
          f"test_acc: {test_acc:.4f}"
        )

        # Update results dictionary
        results["train_loss"].append(train_loss)
        results["train_acc"].append(train_acc)
        results["test_loss"].append(test_loss)
        results["test_acc"].append(test_acc)
        
        
        
        ### New: Experiment tracking ###
        # Add loss results to SummaryWriter

    #     writer.add_scalars(main_tag="Loss",
    #     tag_scalar_dict={"train_loss":train_loss,
    #     "test_loss":test_loss},
    #     global_step=epoch
    #     )
    #     writer.add_scalars(main_tag="Accuracy",
    #     tag_scalar_dict={"train_acc":train_acc,
    #     "test_acc":test_acc},
    #     global_step=epoch
    #     )

    #     writer.add_graph(model=model,
    #                     # Pass in an example input
    #                     input_to_model=torch.randn(32,3,224,224).to(device)
    #     )
        
    # writer.close()

        ### New: Use the writer parameter to track experiments ###
        # See if there's a writer, if so, log to it
        if writer:
            # Add results to SummaryWriter
            writer.add_scalars(main_tag="Loss", 
                               tag_scalar_dict={"train_loss": train_loss,
                                                "test_loss": test_loss},
                               global_step=epoch)
            writer.add_scalars(main_tag="Accuracy", 
                               tag_scalar_dict={"train_acc": train_acc,
                                                "test_acc": test_acc}, 
                               global_step=epoch)

            # Close the writer
            writer.close()
        else:
            pass
    ### End new ###

    return results


In [27]:
# Note: Not using engine.train() since the original script isn't updated to use writer

# Create an example writer
example_writer = create_writer(experiment_name="data_10_percent",
                               model_name="effnetb0",
                               extra="5_epochs")

results = train(model=model,
                train_dataloader=train_dataloader,
                test_dataloader=test_dataloader,
                optimizer=optim,
                loss_fn=loss_fn,
                epochs=10,
                writer=example_writer,
                device=device)

[INFO] Created SummaryWriter, saving to: runs\2024-04-08\data_10_percent\effnetb0\5_epochs...


  0%|          | 0/10 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 0.0754 | train_acc: 0.9896 | test_loss: 0.5525 | test_acc: 0.7773
Epoch: 2 | train_loss: 0.0553 | train_acc: 1.0000 | test_loss: 0.6386 | test_acc: 0.7617
Epoch: 3 | train_loss: 0.1137 | train_acc: 0.9394 | test_loss: 0.7224 | test_acc: 0.7461
Epoch: 4 | train_loss: 0.0729 | train_acc: 0.9697 | test_loss: 0.6236 | test_acc: 0.7500
Epoch: 5 | train_loss: 0.0099 | train_acc: 1.0000 | test_loss: 0.6091 | test_acc: 0.7461
Epoch: 6 | train_loss: 0.0275 | train_acc: 1.0000 | test_loss: 0.5944 | test_acc: 0.7422
Epoch: 7 | train_loss: 0.0289 | train_acc: 1.0000 | test_loss: 0.5924 | test_acc: 0.7383
Epoch: 8 | train_loss: 0.1230 | train_acc: 0.9697 | test_loss: 0.6076 | test_acc: 0.7383
Epoch: 9 | train_loss: 0.0279 | train_acc: 1.0000 | test_loss: 0.7505 | test_acc: 0.7422
Epoch: 10 | train_loss: 0.0242 | train_acc: 1.0000 | test_loss: 0.7920 | test_acc: 0.7461


In [29]:
# !taskkill /PID 1948 /F

# %load_ext tensorboard

%reload_ext tensorboard
%tensorboard --logdir runs --port 6007

Reusing TensorBoard on port 6007 (pid 18044), started 0:17:55 ago. (Use '!kill 18044' to kill it.)

In [35]:
def download_data(source: str, 
                  destination: str,
                  remove_source: bool = True) -> Path:
    """Downloads a zipped dataset from source and unzips to destination.

    Args:
        source (str): A link to a zipped file containing data.
        destination (str): A target directory to unzip data to.
        remove_source (bool): Whether to remove the source after downloading and extracting.
    
    Returns:
        pathlib.Path to downloaded data.
    
    Example usage:
        download_data(source="https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip",
                      destination="pizza_steak_sushi")
    """
    # Setup path to data folder
    data_path = Path("../data/")
    image_path = data_path / destination

    # If the image folder doesn't exist, download it and prepare it... 
    if image_path.is_dir():
        print(f"[INFO] {image_path} directory exists, skipping download.")
    else:
        print(f"[INFO] Did not find {image_path} directory, creating one...")
        image_path.mkdir(parents=True, exist_ok=True)
        
        # Download pizza, steak, sushi data
        target_file = Path(source).name
        with open(data_path / target_file, "wb") as f:
            request = requests.get(source)
            print(f"[INFO] Downloading {target_file} from {source}...")
            f.write(request.content)

        # Unzip pizza, steak, sushi data
        with zipfile.ZipFile(data_path / target_file, "r") as zip_ref:
            print(f"[INFO] Unzipping {target_file} data...") 
            zip_ref.extractall(image_path)

        # Remove .zip file
        if remove_source:
            os.remove(data_path / target_file)
    
    return image_path



Experiment number	Training Dataset	                Model (pretrained on ImageNet)	Number of epochs
1	                Pizza, Steak, Sushi 10% percent	    EfficientNetB0	                5
2	                Pizza, Steak, Sushi 10% percent	    EfficientNetB2	                5
3	                Pizza, Steak, Sushi 10% percent	    EfficientNetB0	                10
4	                Pizza, Steak, Sushi 10% percent	    EfficientNetB2	                10
5	                Pizza, Steak, Sushi 20% percent	    EfficientNetB0	                5
6	                Pizza, Steak, Sushi 20% percent	    EfficientNetB2	                5
7	                Pizza, Steak, Sushi 20% percent	    EfficientNetB0	                10
8	                Pizza, Steak, Sushi 20% percent	    EfficientNetB2	                10

In [38]:
data_10_percent_path = download_data(source="https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip",
                                     destination="pizza_steak_sushi")
# Downloading data 20% for food101
data_20_percent_path = download_data(source="https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi_20_percent.zip",
                                     destination="pizza_steak_sushi_20_percent")

# Setup training directory paths
train_dir_10_percent = data_10_percent_path / "train"
train_dir_20_percent = data_20_percent_path / "train"

# Setup testing directory paths (note: use the same test dataset for both to compare the results)
test_dir = data_10_percent_path / "test"

# Check the directories
print(f"Training directory 10%: {train_dir_10_percent}")
print(f"Training directory 20%: {train_dir_20_percent}")
print(f"Testing directory: {test_dir}")


[INFO] ..\data\pizza_steak_sushi directory exists, skipping download.
[INFO] ..\data\pizza_steak_sushi_20_percent directory exists, skipping download.
Training directory 10%: ..\data\pizza_steak_sushi\train
Training directory 20%: ..\data\pizza_steak_sushi_20_percent\train
Testing directory: ..\data\pizza_steak_sushi\test


In [43]:
# Create a transform to normalize data distribution to be inline with ImageNet
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], # values per colour channel [red, green, blue]
                                 std=[0.229, 0.224, 0.225]) # values per colour channel [red, green, blue]

# Compose transforms into a pipeline
simple_transform = transforms.Compose([
    transforms.Resize((224, 224)), # 1. Resize the images
    transforms.ToTensor(), # 2. Turn the images into tensors with values between 0 & 1
    normalize # 3. Normalize the images so their distributions match the ImageNet dataset 
])

BATCH_SIZE = 32

# Create 10% training and test DataLoaders
train_dataloader_10_percent, test_dataloader, class_names = data_setup.create_dataloaders(train_dir=train_dir_10_percent,
    test_dir=test_dir, 
    transform=simple_transform,
    batch_size=BATCH_SIZE
)

# Create 20% training and test data DataLoders
train_dataloader_20_percent, test_dataloader, class_names = data_setup.create_dataloaders(train_dir=train_dir_20_percent,
    test_dir=test_dir,
    transform=simple_transform,
    batch_size=BATCH_SIZE
)

In [45]:
# 1. Create an instance of EffNetB2 with pretrained weights
effnetb2_weights = torchvision.models.EfficientNet_B2_Weights.DEFAULT # "DEFAULT" means best available weights
effnetb2 = torchvision.models.efficientnet_b2(weights=effnetb2_weights)
print(effnetb2.state_dict)

# 2. Get a summary of standard EffNetB2 from torchvision.models 
summary(model=effnetb2, 
        input_size=(32, 3, 224, 224), # make sure this is "input_size", not "input_shape"
        # col_names=["input_size"], # uncomment for smaller output
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"]
) 


<bound method Module.state_dict of EfficientNet(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): MBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
            (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
            (activation): SiLU(inplace=True)
            (scale_activation): Sigmoid()
         

Layer (type (var_name))                                      Input Shape          Output Shape         Param #              Trainable
EfficientNet (EfficientNet)                                  [32, 3, 224, 224]    [32, 1000]           --                   True
├─Sequential (features)                                      [32, 3, 224, 224]    [32, 1408, 7, 7]     --                   True
│    └─Conv2dNormActivation (0)                              [32, 3, 224, 224]    [32, 32, 112, 112]   --                   True
│    │    └─Conv2d (0)                                       [32, 3, 224, 224]    [32, 32, 112, 112]   864                  True
│    │    └─BatchNorm2d (1)                                  [32, 32, 112, 112]   [32, 32, 112, 112]   64                   True
│    │    └─SiLU (2)                                         [32, 32, 112, 112]   [32, 32, 112, 112]   --                   --
│    └─Sequential (1)                                        [32, 32, 112, 112]   [32, 16, 112

In [50]:
import torchvision
from torch import nn

# Get num out features (one for each class pizza, steak, sushi)
OUT_FEATURES = len(class_names)

# Create an EffNetB0 feature extractor
def create_effnetb0():
    # 1. Get the base mdoel with pretrained weights and send to target device
    weights = torchvision.models.EfficientNet_B0_Weights.DEFAULT
    model = torchvision.models.efficientnet_b0(weights=weights).to(device)

    # 2. Freeze the base model layers
    for param in model.features.parameters():
        param.requires_grad = False



    # 4. Change the classifier head
    model.classifier = nn.Sequential(
        nn.Dropout(p=0.2),
        nn.Linear(in_features=1280, out_features=OUT_FEATURES)
    ).to(device)

    # 5. Give the model a name
    model.name = "effnetb0"
    print(f"[INFO] Created new {model.name} model.")
    return model

# Create an EffNetB2 feature extractor
def create_effnetb2():
    # 1. Get the base model with pretrained weights and send to target device
    weights = torchvision.models.EfficientNet_B2_Weights.DEFAULT
    model = torchvision.models.efficientnet_b2(weights=weights).to(device)

    # 2. Freeze the base model layers
    for param in model.features.parameters():
        param.requires_grad = False



    # 4. Change the classifier head
    model.classifier = nn.Sequential(
        nn.Dropout(p=0.3),
        nn.Linear(in_features=1408, out_features=OUT_FEATURES)
    ).to(device)

    # 5. Give the model a name
    model.name = "effnetb2"
    print(f"[INFO] Created new {model.name} model.")
    return model

In [51]:
effnetb0 = create_effnetb0() 

summary(model=effnetb0, 
        input_size=(32, 3, 224, 224), # make sure this is "input_size", not "input_shape"
        # col_names=["input_size"], # uncomment for smaller output
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"]
)

[INFO] Created new effnetb0 model.


Layer (type (var_name))                                      Input Shape          Output Shape         Param #              Trainable
EfficientNet (EfficientNet)                                  [32, 3, 224, 224]    [32, 3]              --                   Partial
├─Sequential (features)                                      [32, 3, 224, 224]    [32, 1280, 7, 7]     --                   False
│    └─Conv2dNormActivation (0)                              [32, 3, 224, 224]    [32, 32, 112, 112]   --                   False
│    │    └─Conv2d (0)                                       [32, 3, 224, 224]    [32, 32, 112, 112]   (864)                False
│    │    └─BatchNorm2d (1)                                  [32, 32, 112, 112]   [32, 32, 112, 112]   (64)                 False
│    │    └─SiLU (2)                                         [32, 32, 112, 112]   [32, 32, 112, 112]   --                   --
│    └─Sequential (1)                                        [32, 32, 112, 112]   [32, 

In [52]:
effnetb2 = create_effnetb2()

summary(model=effnetb2, 
        input_size=(32, 3, 224, 224), # make sure this is "input_size", not "input_shape"
        # col_names=["input_size"], # uncomment for smaller output
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"]
)

[INFO] Created new effnetb2 model.


Layer (type (var_name))                                      Input Shape          Output Shape         Param #              Trainable
EfficientNet (EfficientNet)                                  [32, 3, 224, 224]    [32, 3]              --                   Partial
├─Sequential (features)                                      [32, 3, 224, 224]    [32, 1408, 7, 7]     --                   False
│    └─Conv2dNormActivation (0)                              [32, 3, 224, 224]    [32, 32, 112, 112]   --                   False
│    │    └─Conv2d (0)                                       [32, 3, 224, 224]    [32, 32, 112, 112]   (864)                False
│    │    └─BatchNorm2d (1)                                  [32, 32, 112, 112]   [32, 32, 112, 112]   (64)                 False
│    │    └─SiLU (2)                                         [32, 32, 112, 112]   [32, 32, 112, 112]   --                   --
│    └─Sequential (1)                                        [32, 32, 112, 112]   [32, 

In [53]:
# 1. Create epochs list
num_epochs = [5, 10]

# 2. Create models list (need to create a new model for each experiment)
models = ["effnetb0", "effnetb2"]

# 3. Create dataloaders dictionary for various dataloaders
train_dataloaders = {"data_10_percent": train_dataloader_10_percent,
                     "data_20_percent": train_dataloader_20_percent}

In [55]:
%%time
from GoingModular.utils import save_model

# 1. Set the random seeds


# 2. Keep track of experiment numbers
experiment_number = 0

# 3. Loop through each DataLoader
for dataloader_name, train_dataloader in train_dataloaders.items():

    # 4. Loop through each number of epochs
    for epochs in num_epochs: 

        # 5. Loop through each model name and create a new model based on the name
        for model_name in models:

            # 6. Create information print outs
            experiment_number += 1
            print(f"[INFO] Experiment number: {experiment_number}")
            print(f"[INFO] Model: {model_name}")
            print(f"[INFO] DataLoader: {dataloader_name}")
            print(f"[INFO] Number of epochs: {epochs}")  

            # 7. Select the model
            if model_name == "effnetb0":
                model = create_effnetb0() # creates a new model each time (important because we want each experiment to start from scratch)
            else:
                model = create_effnetb2() # creates a new model each time (important because we want each experiment to start from scratch)
            
            # 8. Create a new loss and optimizer for every model
            loss_fn = nn.CrossEntropyLoss()
            optimizer = torch.optim.Adam(params=model.parameters(), lr=0.001)

            # 9. Train target model with target dataloaders and track experiments
            train(model=model,
                  train_dataloader=train_dataloader,
                  test_dataloader=test_dataloader, 
                  optimizer=optimizer,
                  loss_fn=loss_fn,
                  epochs=epochs,
                  device=device,
                  writer=create_writer(experiment_name=dataloader_name,
                                       model_name=model_name,
                                       extra=f"{epochs}_epochs"))
            
            # 10. Save the model to file so we can get back the best model
            save_filepath = f"07_{model_name}_{dataloader_name}_{epochs}_epochs.pth"
            save_model(model=model,
                       target_dir="models",
                       model_name=save_filepath)
            print("-"*50 + "\n")

[INFO] Experiment number: 1
[INFO] Model: effnetb0
[INFO] DataLoader: data_10_percent
[INFO] Number of epochs: 5
[INFO] Created new effnetb0 model.
[INFO] Created SummaryWriter, saving to: runs\2024-04-08\data_10_percent\effnetb0\5_epochs...


  0%|          | 0/5 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 1.0144 | train_acc: 0.5039 | test_loss: 0.7912 | test_acc: 0.8958
Epoch: 2 | train_loss: 0.8587 | train_acc: 0.7500 | test_loss: 0.6982 | test_acc: 0.9479
Epoch: 3 | train_loss: 0.7447 | train_acc: 0.7500 | test_loss: 0.5930 | test_acc: 0.8854
Epoch: 4 | train_loss: 0.6723 | train_acc: 0.7695 | test_loss: 0.6060 | test_acc: 0.8864
Epoch: 5 | train_loss: 0.6086 | train_acc: 0.7617 | test_loss: 0.5412 | test_acc: 0.8864
[INFO] Saving model to: models\07_effnetb0_data_10_percent_5_epochs.pth
--------------------------------------------------

[INFO] Experiment number: 2
[INFO] Model: effnetb2
[INFO] DataLoader: data_10_percent
[INFO] Number of epochs: 5
[INFO] Created new effnetb2 model.
[INFO] Created SummaryWriter, saving to: runs\2024-04-08\data_10_percent\effnetb2\5_epochs...


  0%|          | 0/5 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 1.1006 | train_acc: 0.3594 | test_loss: 1.0433 | test_acc: 0.5066
Epoch: 2 | train_loss: 0.9034 | train_acc: 0.6562 | test_loss: 0.9269 | test_acc: 0.6610
Epoch: 3 | train_loss: 0.7732 | train_acc: 0.8789 | test_loss: 0.7877 | test_acc: 0.8239
Epoch: 4 | train_loss: 0.6980 | train_acc: 0.8086 | test_loss: 0.7138 | test_acc: 0.8759
Epoch: 5 | train_loss: 0.6434 | train_acc: 0.8281 | test_loss: 0.6976 | test_acc: 0.8352
[INFO] Saving model to: models\07_effnetb2_data_10_percent_5_epochs.pth
--------------------------------------------------

[INFO] Experiment number: 3
[INFO] Model: effnetb0
[INFO] DataLoader: data_10_percent
[INFO] Number of epochs: 10
[INFO] Created new effnetb0 model.
[INFO] Created SummaryWriter, saving to: runs\2024-04-08\data_10_percent\effnetb0\10_epochs...


...

Epoch: 1 | train_loss: 0.9678 | train_acc: 0.5771 | test_loss: 0.7320 | test_acc: 0.9479
Epoch: 2 | train_loss: 0.7055 | train_acc: 0.8208 | test_loss: 0.6405 | test_acc: 0.8769
Epoch: 3 | train_loss: 0.5969 | train_acc: 0.8042 | test_loss: 0.5736 | test_acc: 0.8665
Epoch: 4 | train_loss: 0.4866 | train_acc: 0.9021 | test_loss: 0.4842 | test_acc: 0.9176
Epoch: 5 | train_loss: 0.4777 | train_acc: 0.8292 | test_loss: 0.4837 | test_acc: 0.8665
[INFO] Saving model to: models\07_effnetb2_data_20_percent_5_epochs.pth
--------------------------------------------------

[INFO] Experiment number: 7
[INFO] Model: effnetb0
[INFO] DataLoader: data_20_percent
[INFO] Number of epochs: 10
[INFO] Created new effnetb0 model.
[INFO] Created SummaryWriter, saving to: runs\2024-04-08\data_20_percent\effnetb0\10_epochs...


  0%|          | 0/10 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 0.9771 | train_acc: 0.5729 | test_loss: 0.7511 | test_acc: 0.7945
Epoch: 2 | train_loss: 0.7075 | train_acc: 0.8063 | test_loss: 0.5954 | test_acc: 0.8977
Epoch: 3 | train_loss: 0.5473 | train_acc: 0.8646 | test_loss: 0.4835 | test_acc: 0.8873
Epoch: 4 | train_loss: 0.4744 | train_acc: 0.8917 | test_loss: 0.4484 | test_acc: 0.8873
Epoch: 5 | train_loss: 0.4477 | train_acc: 0.8625 | test_loss: 0.3861 | test_acc: 0.8873
Epoch: 6 | train_loss: 0.3891 | train_acc: 0.8688 | test_loss: 0.3877 | test_acc: 0.8873
Epoch: 7 | train_loss: 0.3471 | train_acc: 0.9167 | test_loss: 0.3455 | test_acc: 0.8977
Epoch: 8 | train_loss: 0.3381 | train_acc: 0.8917 | test_loss: 0.3318 | test_acc: 0.9081
Epoch: 9 | train_loss: 0.3139 | train_acc: 0.9062 | test_loss: 0.3481 | test_acc: 0.8977
Epoch: 10 | train_loss: 0.3337 | train_acc: 0.8875 | test_loss: 0.3639 | test_acc: 0.9081
[INFO] Saving model to: models\07_effnetb0_data_20_percent_10_epochs.pth
------------------------------------

  0%|          | 0/10 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 1.0035 | train_acc: 0.5167 | test_loss: 0.8418 | test_acc: 0.7746
Epoch: 2 | train_loss: 0.7223 | train_acc: 0.8146 | test_loss: 0.7033 | test_acc: 0.8163
Epoch: 3 | train_loss: 0.5719 | train_acc: 0.8833 | test_loss: 0.6388 | test_acc: 0.8362
Epoch: 4 | train_loss: 0.5443 | train_acc: 0.8438 | test_loss: 0.5248 | test_acc: 0.9176
Epoch: 5 | train_loss: 0.4566 | train_acc: 0.8938 | test_loss: 0.5037 | test_acc: 0.9072
Epoch: 6 | train_loss: 0.4258 | train_acc: 0.8750 | test_loss: 0.5115 | test_acc: 0.8665
Epoch: 7 | train_loss: 0.4214 | train_acc: 0.8271 | test_loss: 0.4903 | test_acc: 0.8873
Epoch: 8 | train_loss: 0.3516 | train_acc: 0.9104 | test_loss: 0.4390 | test_acc: 0.9176
Epoch: 9 | train_loss: 0.3641 | train_acc: 0.8812 | test_loss: 0.4560 | test_acc: 0.8665
Epoch: 10 | train_loss: 0.3117 | train_acc: 0.9333 | test_loss: 0.4297 | test_acc: 0.8258
[INFO] Saving model to: models\07_effnetb2_data_20_percent_10_epochs.pth
------------------------------------

In [56]:
%reload_ext tensorboard
%tensorboard --logdir runs --port 6007

Reusing TensorBoard on port 6007 (pid 18044), started 1:01:02 ago. (Use '!kill 18044' to kill it.)